<a href="https://colab.research.google.com/github/nextcoders/DrQA/blob/master/llama_3_2_1b_df_pandas-V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install accelerate peft bitsandbytes transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 102.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers

In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [3]:
# Installing More Dependencies
import torch
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
import os

In [4]:
#use this or the code below for generating the 4-bit quantified model

from transformers import AutoTokenizer, AutoModelForCausalLM

# Specify the model name or path from Hugging Face
model_id="meta-llama/Llama-3.2-1B"

# Load the tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [ ]:
# def get_model_and_tokenizer(model_id):
#   tokenizer = AutoTokenizer.from_pretrained(model_id)
#   tokenizer.pad_token = tokenizer.eos_token
#   bnb_config = BitsAndBytesConfig(
#       load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype="float16", bnb_4bit_use_double_quant=True
#   )
#   model = AutoModelForCausalLM.from_pretrained(
#       model_id, quantization_config=bnb_config, device_map="auto"
#   )
#   model.config.use_cache=False
#   model.config.pretraining_tp=1
#   return model, tokenizer

In [ ]:
# model, tokenizer = get_model_and_tokenizer(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/56.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/68.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/700 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [5]:
# Step 1: Install xmltodict if not already installed
!pip install xmltodict

# Step 2: Import required libraries
import xmltodict
import json
from google.colab import files

# Step 3: Upload the XML file
uploaded = files.upload()

# Step 4: Load and convert XML to JSON
for file_name in uploaded.keys():
    # Read the XML file
    with open(file_name, "r") as file:
        xml_data = file.read()

    # Parse XML to dict
    dict_data = xmltodict.parse(xml_data)

    # Convert dict to JSON
    json_data = json.dumps(dict_data, indent=4)

    # Print JSON data
    print(json_data)

    # Optional: Save JSON to file
    json_file_name = file_name.replace(".xml", ".json")
    with open(json_file_name, "w") as json_file:
        json_file.write(json_data)
    print(f"JSON saved to {json_file_name}")

Saving scheme-summary.xml to scheme-summary.xml
{
    "SchemeSummaryDocument": {
        "@xmlns:xsi": "http://www.w3.org/2001/XMLSchema-instance",
        "SchemeSummary": [
            {
                "Fund_Name": "Franklin India Equity Savings Fund",
                "Option_Names_Regular__Direct": "Income Distribution cum capital withdrawal (IDCW) Plan (DP), Monthly IDCW Plan (MD), Quarterly IDCW Plan (QD) and Growth Plan (GP). Direct - IDCW Plan (DP), Direct - Monthly IDCW Plan (MD), Direct - Quarterly IDCW Plan (QD) and Direct - Growth Plan (GP).",
                "Fund_Type": "An open ended scheme investing in equity, arbitrage and debt",
                "Riskometer_At_the_time_of_Launch": "Moderately High",
                "Riskometer_as_on_Date": "Low to Moderate",
                "Category_as_Per_SEBI_Categorization_Circular": "Equity Savings Fund",
                "Potential_Risk_Class_as_on_date": "Not applicable",
                "Description_Objective_of_the_scheme": "Th

In [45]:
# Convert JSON string back to a dictionary
dict_data = json.loads(json_data)

import pandas as pd

# Normalize JSON data
df = pd.json_normalize(dict_data, record_path=['SchemeSummaryDocument', 'SchemeSummary'])

# Access 'Fund_Type' column
fund_types = df['Fund_Type'].tolist()


In [113]:

# keyword = None
def generate_matchingFunds(user_input, keyword):
  user_input = "Find fund type: debt"
  if "type:" in user_input:
    keyword = user_input.split("type:")[1].strip().lower()
    # print(keyword)

  if keyword:
    # Create boolean vector to check if 'debt' exists in Fund_Type
    type_funds_bool = df['Fund_Type'].str.contains(keyword, case=False)
    # Use boolean vector to filter Fund_Name
    type_fund_names = df.loc[type_funds_bool, 'Fund_Name'].tolist()
    matching_funds = type_fund_names
  else:
    fund_bool = df['Fund_Type'].str.contains(keyword, case=False)
    matching_funds = df.loc[fund_bool, 'Fund_Name'].tolist()
    # matching_funds = df['Fund_Name'].tolist()

  if not matching_funds:
    return "No matching fund types found."

  # print(matching_funds)
  # print(len(matching_funds))

  matching_funds_df = pd.DataFrame(matching_funds, columns=['Fund_Name'])
  # print(matching_funds_df)
  return matching_funds_df

# matching_funds=generate_matchingFunds(user_input, keyword)

generate_matchingFunds(user_input, keyword)

,Fund_Name
0,Franklin India Equity Savings Fund
1,Franklin India Overnight Fund
2,Franklin India Ultra Short Duration Fund
3,Franklin India Medium to Long Duration Fund
4,Franklin India Government Securities Fund
5,Franklin India Corporate Debt Fund
6,Franklin India Debt Hybrid Fund (No. of segreg...
7,Franklin India Floating Rate Fund
8,Franklin India Money Market Fund
9,Franklin India Banking & PSU Debt Fund


In [130]:
import re
def generate_response(user_input, matching_funds_df):
# def generate_response(user_input, matching_funds):
  matching_funds_df = generate_matchingFunds(user_input, keyword)

  prompt = f"Print any 5 rows inside {matching_funds_df}"

  # Encode input prompt
  inputs = tokenizer(prompt, return_tensors="pt")

  # Generate response
  output = model.generate(
    inputs["input_ids"],
    attention_mask=inputs["attention_mask"],
    max_new_tokens=120,
    # stop_sequences=["\n\nTop 5 Matching Funds:\n"],
    eos_token_id=tokenizer.eos_token_id
  )

  # Decode generated response
  # response = tokenizer.decode(output[0], skip_special_tokens=True)

  # Include matching funds in response
  #response += f"\n\nMatching Funds: {', '.join(matching_funds_df)}"
  response = f"Top 5 Matching Funds:\n{matching_funds_df.head(5).to_string()}"

  return response

In [131]:
# Example usage:
user_input = "Find fund type: debt"
response = generate_response(user_input, matching_funds_df)
print(response)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Top 5 Matching Funds:
                                     Fund_Name
0           Franklin India Equity Savings Fund
1                Franklin India Overnight Fund
2     Franklin India Ultra Short Duration Fund
3  Franklin India Medium to Long Duration Fund
4    Franklin India Government Securities Fund
